## Scenario 2: A cross-functional team with one data scientist working on an ML model


MLflow setup:
- tracking server: yes, local server
- backend store: sqlite database
- artifacts store: local filesystem

The experiments can be explored locally by accessing the local tracking server.

To run this example you need to launch the mlflow server locally by running the following command in your terminal:

`mlflow server --backend-store-uri sqlite:///backend.db`

In [1]:
import mlflow


mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [2]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://127.0.0.1:5000'


In [3]:
mlflow.search_experiments()


[<Experiment: artifact_location=('/workspaces/mlops-zoomcamp/2. '
  'experiment-tracking/running-mlflow-in-practice/artifacts_local/0'), creation_time=1711472423205, experiment_id='0', last_update_time=1711472423205, lifecycle_stage='active', name='Default', tags={}>]

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2024/03/26 17:01:04 INFO mlflow.tracking.fluent: Experiment with name 'my-experiment-1' does not exist. Creating a new experiment.


default artifacts URI: '/workspaces/mlops-zoomcamp/2. experiment-tracking/running-mlflow-in-practice/artifacts_local/1/5d8ce9ea6a994ad89c7a789907ddb3e6/artifacts'


In [5]:
mlflow.search_experiments()

[<Experiment: artifact_location=('/workspaces/mlops-zoomcamp/2. '
  'experiment-tracking/running-mlflow-in-practice/artifacts_local/1'), creation_time=1711472464864, experiment_id='1', last_update_time=1711472464864, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location=('/workspaces/mlops-zoomcamp/2. '
  'experiment-tracking/running-mlflow-in-practice/artifacts_local/0'), creation_time=1711472423205, experiment_id='0', last_update_time=1711472423205, lifecycle_stage='active', name='Default', tags={}>]

### Interacting with the model registry

In [6]:
from mlflow.tracking import MlflowClient


client = MlflowClient("http://127.0.0.1:5000")

In [7]:
client.search_registered_models()             # search registered model in the registry

[]

In [9]:
#run_id = client.list_runs_infos(experiment_id='1')[0].run_id      # check back for this

run_id = '5d8ce9ea6a994ad89c7a789907ddb3e6'
model_uri = f'runs:/{run_id}/models'

mlflow.register_model(
    model_uri=model_uri,
    name='iris-classifier'
)

Registered model 'iris-classifier' already exists. Creating a new version of this model...
2024/03/26 17:02:37 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris-classifier, version 1
Created version '1' of model 'iris-classifier'.


<ModelVersion: aliases=[], creation_timestamp=1711472557862, current_stage='None', description='', last_updated_timestamp=1711472557862, name='iris-classifier', run_id='5d8ce9ea6a994ad89c7a789907ddb3e6', run_link='', source=('/workspaces/mlops-zoomcamp/2. '
 'experiment-tracking/running-mlflow-in-practice/artifacts_local/1/5d8ce9ea6a994ad89c7a789907ddb3e6/artifacts/models'), status='READY', status_message='', tags={}, user_id='', version='1'>

In [10]:
client.transition_model_version_stage(
    name='iris-classifier',
    version=1,
    stage='Production',
    archive_existing_versions=False
)

/tmp/ipykernel_3136/233037838.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.11.1/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1711472557862, current_stage='Production', description='', last_updated_timestamp=1711472603790, name='iris-classifier', run_id='5d8ce9ea6a994ad89c7a789907ddb3e6', run_link='', source=('/workspaces/mlops-zoomcamp/2. '
 'experiment-tracking/running-mlflow-in-practice/artifacts_local/1/5d8ce9ea6a994ad89c7a789907ddb3e6/artifacts/models'), status='READY', status_message='', tags={}, user_id='', version='1'>

In [11]:
client.update_model_version(
    name='iris-classifier',
    version=1,
    description='this model is good for production'

)

<ModelVersion: aliases=[], creation_timestamp=1711472557862, current_stage='Production', description='this model is good for production', last_updated_timestamp=1711472607967, name='iris-classifier', run_id='5d8ce9ea6a994ad89c7a789907ddb3e6', run_link='', source=('/workspaces/mlops-zoomcamp/2. '
 'experiment-tracking/running-mlflow-in-practice/artifacts_local/1/5d8ce9ea6a994ad89c7a789907ddb3e6/artifacts/models'), status='READY', status_message='', tags={}, user_id='', version='1'>